In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

vector_store = Chroma(collection_name="income_tax_collection",
                      embedding_function=embeddings,
                      persist_directory="./income_tax_collection")

retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [3]:
from langchain_upstage import ChatUpstage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: list
    answer: str

graph_builder = StateGraph(AgentState)
llm = ChatUpstage()

In [4]:
def retrieve(state: AgentState) -> AgentState:
    query = state['query']
    docs = retriever.invoke(query)
    print(f"retrieve: {docs}")
    return {'context': docs}

In [ ]:
from typing import Literal
from langchain import hub
from langchain_core.prompts import PromptTemplate

doc_relevance_prompt = hub.pull("langchain-ai/rag-document-relevance")

def check_doc_relevance(state: AgentState) -> Literal['relevant', 'irrelevant']:
    query = state['query']
    context = state['context']
    print(f"check_doc_relevance: {query}{context}")
    doc_relevance_chain = doc_relevance_prompt | llm
    response = doc_relevance_chain.invoke({'question': query, 'documents': context})
    print(f"check_doc_relevance: {response}")
    if response:
        if response['Score'] == 1:
            return 'relevant'
        else:
            return 'irrelevant'
    else:
        return 'irrelevant'

c:\Users\hyunseo\Desktop\learn_langgraph\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [6]:
from langchain import hub
from langchain_upstage import ChatUpstage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

generate_prompt = PromptTemplate.from_template(f"""
    [Identity]
    "당신은 소득세법 전문가입니다. 사용자의 소득세법에 관한 질문에 답변해주세요"
    "아래에 제공된 문서를 활용해서 답변해주시고"
    "답변을 알 수 없다면 모른다고 답변해주세요"
    "특히 소득세법 전문가이기 때문에 수학계산을 정확히 해주시길 바랍니다."
    "답변을 제공할 때는 소득세법 (XX조)에 따르면 이라고 시작하면서 답변해주세요."
        
    [Context]
    {{context}}
    
    Question: {{question}}
""" 
)

def generate(state: AgentState) -> AgentState:
    query = state['query']
    context = state['context']
    # 이곳에 StrOutputParser를 붙이지 않으면 rag_chain의 결과가 llm의 답변인 Message 객체가 됨
    # 하지만 우리 state에서 'answer'의 type은 str이므로 str로 바꿔줘야함
    # 여기서는 바꾸지 않아도 에러가 안나지만 에러가 날 때도 있다고 한다. 그건 나중에 공부해보자자
    rag_chain = generate_prompt | llm | StrOutputParser()
    response = rag_chain.invoke({'question': query, 'context': context})
    print(f"generate: {response}")
    return {'answer': response}

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

rewrite_prompt = PromptTemplate.from_template(f"""
    사용자의 질문을 보고, 웹 검색에 용이하게 사용자의 질문을 수정해주세요.
    답변은 **오직 수정된 질문만** 리턴해주세요.
    질문: {{query}}
    """
)

def rewrite(state: AgentState) -> AgentState:
    query = state['query']
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()
    response = rewrite_chain.invoke({'query': query})
    print(f"rewrite: {response}")
    return {'query': response}

In [8]:
from langchain_tavily import TavilySearch

tavily_search_tool = TavilySearch(
    max_results=3,
    topic="general",
)

def web_search(state: AgentState) -> AgentState:
    query = state['query']
    results = tavily_search_tool.invoke(query)
    print(f"web_search: {results}")
    return {'context': results}

In [9]:
graph_builder.add_node('retrieve', retrieve)
graph_builder.add_node('generate', generate)
graph_builder.add_node('rewrite', rewrite)
graph_builder.add_node('web_search', web_search)

In [10]:
from langgraph.graph import START, END

graph_builder.add_edge(START, 'retrieve')
graph_builder.add_conditional_edges(
    'retrieve',
    check_doc_relevance,
    {'relevant': 'generate', 
     'irrelevant': 'rewrite'}
)
graph_builder.add_edge('generate', END)
graph_builder.add_edge('rewrite', 'web_search')
graph_builder.add_edge('web_search', 'generate')


In [11]:
# from IPython.display import Image, display

graph = graph_builder.compile()

# display(Image(graph.get_graph().draw_mermaid_png()))

In [12]:
query = "역삼역 맛집을 추천해주세요."
initial_state = {'query': query}
graph.invoke(initial_state)

retrieve: [Document(id='97139ac5-ced4-4c42-a413-2854df6f98b3', metadata={'source': './documents/income_tax.txt'}, page_content='범죄처 122 국가법정정보센터\n한다. <개정 2009. 12. 31, 2010. 12. 27.>\n⑥ 사업자(법인은 포함한다. 이하 이 약에서 같다)가 음식, 숙박용역이나 서비스용역을 공급하고 그 대가를 받을 때 지정제한법에 따른 봉사료를 포함하여 하여야 소득세가 그 사업자에게 대한 소득세를 원천징수하여야 한다. <개정 2009. 12. 31, 2010. 12. 27.>\n⑦ 제정부칙 제향까지의 규정에 이어야 할 자를 “원천징수의무자”라 한다. <개정 2010. 12. 27.>\n⑧ 원천징수의무자의 범위 등 밖에 필요한 사항을 대통령령으로 정한다. <신설 2013. 1. 1.> \n[제목개정 2009. 12. 31]\n제127조(원천징수의무) \n국내에서 거주자나 비거주자에게 다음 각 호의 어느 하나에 해당하는 소득을 지급하는 자(제3호 또는 제4호의 소득을 지급하는 자의 경우에는 사업자)는 대통령령으로 정하는 바에 의하여 그 사업자가 그 소득세를 원천징수하여야 한다. <개정 2009. 12. 31, 2010. 12. 27, 2015. 12. 15, 2020. 12. 29.>\n1. 이자소득\n2. 배당소득\n3. 대여금융소득을 정의하는 사업소득(이하 “원천징수대상 사업소득”이라 한다)\n4. 근로소득. 다만, 다음 각 목의 어느 하나에 해당하는 소득은 제외한다.\n  가. 외국에서 은퇴연금에 주도록 지정한(미국의 제정법으로부터 받는 근로소득\n  나. 국외에 있는 비거주자 또는 규격법인(국내법인 또는 국내영업소를 지정하지 아니하게)은 다만, 다음의 어느 하나에 해당하는 소득은 제외한다.\n  1) 제120조제1항 및 제122항의 비거주자와 국내사업자가 “법인세법” 제94조제1항 및 제126항의 외국법인의 국내사업장에 있었던 원천징수

{'query': '"역삼역 근처 맛집 추천 및 정보"',
 'context': 'No search results found for \'"역삼역 근처 맛집 추천 및 정보"\'. Suggestions: Try a more detailed search using \'advanced\' search_depth. Try modifying your search parameters with one of these approaches.',
 'answer': '소득세법 (XX조)에 따르면, 역삼역 근처 맛집 추천 및 정보에 대한 질문은 소득세법과는 관련이 없는 내용입니다. 소득세법은 주로 개인의 소득에 대한 과세 및 납세에 관한 내용을 다루고 있습니다. 따라서, 본 질문에 대한 답변은 소득세법 전문가의 업무 범위를 벗어납니다. 맛집 추천 및 정보를 원하신다면, 여행이나 음식 관련 커뮤니티나 인터넷 검색을 통해 더 정확한 정보를 얻으실 수 있을 것입니다.'}

In [ ]:
# web-serach를 위한 rewrite는 사실 필요가 없을 수 있다
# 서비스를 만들 때 이 노드가 꼭 필요한 노드인가 고민하는 것은 중요하다!!